In [1]:
import os
import librosa
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from keras.callbacks import EarlyStopping
from keras.regularizers import l2
from keras.optimizers import Adamax

data_path = 'C:\_PROJECT_\Speech Recognition\Data'
class_labels = os.listdir(data_path)

sampling_rate =25050  
n_mfcc = 20 
hop_length = 550  

features = []
labels = []

for label in class_labels:
    label_path = os.path.join(data_path, label)
    for audio_file in os.listdir(label_path):
        audio, _ = librosa.load(os.path.join(label_path, audio_file), sr=sampling_rate)
        mfccs = librosa.feature.mfcc(y=audio, sr=sampling_rate, n_mfcc=n_mfcc, hop_length=hop_length)
        features.append(mfccs.T)  
        labels.append(label)

label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

X_train, X_test, y_train, y_test = train_test_split(features, encoded_labels, test_size=0.2, random_state=42)



In [2]:
X_test=np.array(X_test)
y_test=np.array(y_test)
y_train=np.array(y_train)
X_train=np.array(X_train)

C:\Users\sherm\AppData\Local\Temp\ipykernel_5320\954492346.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_test=np.array(X_test)
C:\Users\sherm\AppData\Local\Temp\ipykernel_5320\954492346.py:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_train=np.array(X_train)


In [3]:
m =0
n=0
for i in range(X_train.shape[0]):
    if(X_train[i].shape[0]>m):
        m=X_train[i].shape[0]
    if(X_train[i].shape[1]>n):
        n=X_train[i].shape[1]

In [4]:

max_shape = (m, n)  
X_train_processed = []
X_test_processed = []

for sample in X_train:
    if sample.shape[0] < max_shape[0]:
        padding = np.zeros((max_shape[0] - sample.shape[0], max_shape[1]))
        processed_sample = np.vstack((sample, padding))
    else:
        processed_sample = sample[:max_shape[0], :]

    X_train_processed.append(processed_sample)

for sample in X_test:
    if sample.shape[0] < max_shape[0]:
        padding = np.zeros((max_shape[0] - sample.shape[0], max_shape[1]))
        processed_sample = np.vstack((sample, padding))
    else:
        processed_sample = sample[:max_shape[0], :]

    X_test_processed.append(processed_sample)

X_train_processed = np.array(X_train_processed)
X_test_processed = np.array(X_test_processed)



In [41]:
model = Sequential()
from keras.optimizers import Adamax
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(X_train_processed.shape[1], X_train_processed.shape[2], 1)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', kernel_regularizer=l2(0.01)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', kernel_regularizer=l2(0.01)))
model.add(MaxPooling2D(pool_size=(1, 1)))

model.add(Flatten())
model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01))) 
model.add(Dropout(0.5))
model.add(Dense(len(class_labels), activation='softmax')) 

model.compile(loss='sparse_categorical_crossentropy', optimizer=Adamax(), metrics=['accuracy'])

X_train_processed = X_train_processed.reshape(X_train_processed.shape[0], X_train_processed.shape[1], X_train_processed.shape[2], 1)
X_test_processed = X_test_processed.reshape(X_test_processed.shape[0], X_test_processed.shape[1], X_test_processed.shape[2], 1)

early_stopping = EarlyStopping(monitor='val_loss', patience=3)  

history = model.fit(X_train_processed, y_train, validation_data=(X_test_processed, y_test), epochs=30, batch_size=32, callbacks=[early_stopping])



Epoch 1/30
9/9 [==============================] - 1s 92ms/step - loss: 5.4111 - accuracy: 0.0741 - val_loss: 4.9134 - val_accuracy: 0.1176
Epoch 2/30
9/9 [==============================] - 1s 74ms/step - loss: 4.7676 - accuracy: 0.0815 - val_loss: 4.6512 - val_accuracy: 0.0735
Epoch 3/30
9/9 [==============================] - 1s 73ms/step - loss: 4.5725 - accuracy: 0.1296 - val_loss: 4.4780 - val_accuracy: 0.1618
Epoch 4/30
9/9 [==============================] - 1s 78ms/step - loss: 4.4264 - accuracy: 0.1407 - val_loss: 4.3009 - val_accuracy: 0.1912
Epoch 5/30
9/9 [==============================] - 1s 108ms/step - loss: 4.3004 - accuracy: 0.1741 - val_loss: 4.1797 - val_accuracy: 0.2206
Epoch 6/30
9/9 [==============================] - 1s 132ms/step - loss: 4.1480 - accuracy: 0.2222 - val_loss: 4.0224 - val_accuracy: 0.3088
Epoch 7/30
9/9 [==============================] - 1s 128ms/step - loss: 3.9320 - accuracy: 0.2407 - val_loss: 3.8894 - val_accuracy: 0.2941
Epoch 8/30
9/9 [========

In [68]:
early_stopping = EarlyStopping(monitor='val_loss', patience=1)
history = model.fit(X_train_processed, y_train, validation_data=(X_test_processed, y_test), epochs=10, batch_size=32, callbacks=[early_stopping])


Epoch 1/10
9/9 [==============================] - 1s 77ms/step - loss: 0.7465 - accuracy: 0.9556 - val_loss: 1.8050 - val_accuracy: 0.7059
Epoch 2/10
9/9 [==============================] - 1s 112ms/step - loss: 0.7439 - accuracy: 0.9481 - val_loss: 1.9982 - val_accuracy: 0.6912


In [39]:
ac=model.evaluate(X_train_processed, y_train)[1]
print(f"Accuracy Close dataset: {ac* 100:f}%")

9/9 [==============================] - 0s 19ms/step - loss: 1.7793 - accuracy: 0.9778
Accuracy Close dataset: 97.777778%


In [60]:
accuracy = model.evaluate(X_test_processed, y_test)[1]
print(f"Accuracy Open dataset: {accuracy * 100:.2f}%")

3/3 [==============================] - 0s 13ms/step - loss: 1.9058 - accuracy: 0.7206
Accuracy Open dataset: 72.06%
